In [2]:
import mne 
import torch 
import torch.nn as nn 


In [14]:
# Load the raw EEG data from a .fif file
raw = mne.io.read_raw_eeglab('/home/technoshr370/Downloads/Dataset/ds003751/sub-mit003/eeg/sub-mit003_task-Emotion_eeg.set', preload=True)

Reading /home/technoshr370/Downloads/Dataset/ds003751/sub-mit003/eeg/sub-mit003_task-Emotion_eeg.fdt
Reading 0 ... 674089  =      0.000 ...  2696.356 secs...


/tmp/ipykernel_57050/956615108.py:2: RuntimeWarning: Unknown types found, setting as type EEG:
pns: ['ECG', 'EMG', 'EMG_2']
  raw = mne.io.read_raw_eeglab('/home/technoshr370/Downloads/Dataset/ds003751/sub-mit003/eeg/sub-mit003_task-Emotion_eeg.set', preload=True)
/tmp/ipykernel_57050/956615108.py:2: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab('/home/technoshr370/Downloads/Dataset/ds003751/sub-mit003/eeg/sub-mit003_task-Emotion_eeg.set', preload=True)


In [15]:
print(raw.info)

<Info | 8 non-empty values
 bads: []
 ch_names: E1, E2, E3, E4, E5, E6, E7, E8, E9, E10, E11, E12, E13, E14, ...
 chs: 132 EEG
 custom_ref_applied: False
 dig: 135 items (3 Cardinal, 132 EEG)
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 132
 projs: []
 sfreq: 250.0 Hz
>


In [22]:
raw.describe()

<RawEEGLAB | sub-mit003_task-Emotion_eeg.fdt, 132 x 674090 (2696.4 s), ~679.0 MB, data loaded>
 ch  name   type  unit        min         Q1     median         Q3        max
  0  E1     EEG   µV    -417694.12     -10.94       0.19      11.25  308440.47
  1  E2     EEG   µV    -372481.89      -9.46      -0.55       8.39  391511.94
  2  E3     EEG   µV    -345493.41      -9.20      -0.70       7.66  446523.12
  3  E4     EEG   µV    -370456.13      -7.32      -0.47       6.25  310315.62
  4  E5     EEG   µV    -272627.91      -6.58      -0.55       5.28   77193.84
  5  E6     EEG   µV    -109008.75      -4.93      -0.39       4.06   74347.87
  6  E7     EEG   µV    -109019.61      -2.98      -0.14       2.76   77742.19
  7  E8     EEG   µV    -389807.12     -17.76      -2.17      11.11  482364.06
  8  E9     EEG   µV    -561977.44     -10.75      -1.16       7.63  492984.37
  9  E10    EEG   µV    -365084.80      -8.22      -0.77       6.33  479000.54
 10  E11    EEG   µV    -210925.13   

In [23]:
# Apply band-pass filter (e.g., 1-40 Hz)
raw.filter(l_freq=1.0, h_freq=40.0)

# Check for events (triggers) in the data
events, event_id = mne.events_from_annotations(raw)

# If events are found, create epochs around them
if len(events) > 0:
    # Define epochs around events
    epochs = mne.Epochs(raw, events=events, event_id=event_id,
                        tmin=-0.2, tmax=0.5, baseline=(None, 0))
else:
    # If no events, create fixed-length epochs
    epochs = mne.make_fixed_length_epochs(raw, duration=1.0, overlap=0.5)

# Get the data array
data = epochs.get_data()  # Shape: (n_epochs, n_channels, n_times)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 825 samples (3.300 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  71 tasks      | elapsed:    0.9s


Used Annotations descriptions: [np.str_('IBEG'), np.str_('IEND'), np.str_('arsl'), np.str_('base'), np.str_('bend'), np.str_('boundary'), np.str_('cate'), np.str_('clic'), np.str_('dmns'), np.str_('fixL'), np.str_('fmrt'), np.str_('lkng'), np.str_('net'), np.str_('puse'), np.str_('qdon'), np.str_('quiz'), np.str_('relv'), np.str_('rsrt'), np.str_('stm'), np.str_('stop'), np.str_('trno'), np.str_('vlnc')]
Not setting metadata
130 matching events found
Setting baseline interval to [-0.2, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 130 events and 176 original time points ...
0 bad epochs dropped


In [24]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Get data dimensions
n_epochs, n_channels, n_times = data.shape

# Flatten the data for input to the VAE
data = data.reshape(n_epochs, n_channels * n_times)

# Normalize the data
scaler = StandardScaler()
data = scaler.fit_transform(data)


## Train test split 

In [26]:
# Split the data into training and validation sets
X_train, X_val = train_test_split(data, test_size=0.2, random_state=42)

# main VAE architecture 

In [27]:
import torch
from torch import nn

class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        # Encoder
        self.fc1 = nn.Linear(input_dim, 512)
        self.fc21 = nn.Linear(512, latent_dim)  # Mean
        self.fc22 = nn.Linear(512, latent_dim)  # LogVariance
        # Decoder
        self.fc3 = nn.Linear(latent_dim, 512)
        self.fc4 = nn.Linear(512, input_dim)

    def encode(self, x):
        h1 = torch.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)  # Return mean and logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std  # Sampling

    def decode(self, z):
        h3 = torch.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))  # Reconstruction

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, input_dim))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


In [28]:
# Parameters
input_dim = X_train.shape[1]
latent_dim = 20
batch_size = 64
epochs = 50
learning_rate = 1e-3


In [29]:
model = VAE(input_dim, latent_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Loss Function
def loss_function(recon_x, x, mu, logvar):
    # Reconstruction loss (BCE or MSE)
    recon_loss = nn.functional.mse_loss(recon_x, x, reduction='sum')
    # KL Divergence
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + kl_loss


In [30]:
from torch.utils.data import DataLoader, TensorDataset

In [31]:

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train).float()
train_dataset = TensorDataset(X_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


In [32]:

# Training Loop
for epoch in range(epochs):
    model.train()
    train_loss = 0
    for data_batch in train_loader:
        data_batch = data_batch[0]  # Extract data from tuple
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data_batch)
        loss = loss_function(recon_batch, data_batch, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    avg_loss = train_loss / len(X_train)
    print(f'Epoch {epoch + 1}, Loss: {avg_loss:.4f}')

Epoch 1, Loss: 33672.6185
Epoch 2, Loss: nan
Epoch 3, Loss: nan
Epoch 4, Loss: nan
Epoch 5, Loss: nan
Epoch 6, Loss: nan
Epoch 7, Loss: nan
Epoch 8, Loss: nan
Epoch 9, Loss: nan
Epoch 10, Loss: nan
Epoch 11, Loss: nan
Epoch 12, Loss: nan
Epoch 13, Loss: nan
Epoch 14, Loss: nan
Epoch 15, Loss: nan
Epoch 16, Loss: nan
Epoch 17, Loss: nan
Epoch 18, Loss: nan
Epoch 19, Loss: nan
Epoch 20, Loss: nan
Epoch 21, Loss: nan
Epoch 22, Loss: nan
Epoch 23, Loss: nan
Epoch 24, Loss: nan
Epoch 25, Loss: nan
Epoch 26, Loss: nan
Epoch 27, Loss: nan
Epoch 28, Loss: nan
Epoch 29, Loss: nan
Epoch 30, Loss: nan
Epoch 31, Loss: nan
Epoch 32, Loss: nan
Epoch 33, Loss: nan
Epoch 34, Loss: nan
Epoch 35, Loss: nan
Epoch 36, Loss: nan
Epoch 37, Loss: nan
Epoch 38, Loss: nan
Epoch 39, Loss: nan
Epoch 40, Loss: nan
Epoch 41, Loss: nan
Epoch 42, Loss: nan
Epoch 43, Loss: nan
Epoch 44, Loss: nan
Epoch 45, Loss: nan
Epoch 46, Loss: nan
Epoch 47, Loss: nan
Epoch 48, Loss: nan
Epoch 49, Loss: nan
Epoch 50, Loss: nan


In [33]:
model.eval()
with torch.no_grad():
    # Convert validation data to tensor
    X_val_tensor = torch.tensor(X_val).float()
    # Encode validation data
    mu, logvar = model.encode(X_val_tensor)
    # Use the mean (mu) as the feature representation
    features = mu.numpy()


In [34]:
from sklearn.manifold import TSNE

# Reduce dimensions for visualization
tsne = TSNE(n_components=2, random_state=42)
features_2d = tsne.fit_transform(features)

plt.scatter(features_2d[:, 0], features_2d[:, 1])
plt.title('t-SNE Visualization of VAE Features')
plt.show()


ValueError: perplexity must be less than n_samples